# SVM como clasificador

## SVM lineal
En *Scikit Learn* pueden encontrarse tres implementaciones distintas del algoritmo Suport Vector Machine:
*   Las clases `sklearn.svm.SVC` y `sklearn.svm.NuSVC` permiten crear modelos SVM de clasificación empleando kernel lineal, polinomial, radial o sigmoide. La diferencia es que `SVC` controla la regularización a través del hiperparámetro `C`, mientras que `NuSVC` lo hace con el número máximo de vectores soporte permitidos.
*  La clase `sklearn.svm.LinearSVC` permite ajustar modelos SVM con kernel lineal. Es similar a SVC cuando el parámetro `kernel='linear'`, pero utiliza un algoritmo más rápido.

Las mismas implementaciones están disponibles para regresión en las clases: `sklearn.svm.SVR`, `sklearn.svm.NuSVR` y `sklearn.svm.LinearSVR`.

Se ajusta primero un modelo SVM con kernel lineal y después uno con kernel radial, y se compara la capacidad de cada uno para clasificar correctamente las observaciones.

In [ ]:
# Tratamiento de datos
# ==============================================================================
import pandas as pd
import numpy as np

# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
from mlxtend.plotting import plot_decision_regions

# Preprocesado y modelado
# ==============================================================================
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer

# Configuración matplotlib
# ==============================================================================
plt.rcParams['image.cmap'] = "bwr"
#plt.rcParams['figure.dpi'] = "100"
plt.rcParams['savefig.bbox'] = "tight"
style.use('ggplot') or plt.style.use('ggplot')

# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Datos
# ==============================================================================
datos = pd.read_csv('./SVM.csv')
datos.head(3)

In [ ]:
fig, ax = plt.subplots(figsize=(6,4))
ax.scatter(datos.X1, datos.X2, c=datos.y);
ax.set_title("Datos SVM");

In [ ]:
#from os import X_OK
# División de los datos en train y test
# ==============================================================================
X0 = datos.drop(columns = 'y')
y0 = datos['y']

X_train, X_test, y_train, y_test = train_test_split(
                                        X0,
                                        y0.values.reshape(-1,1),
                                        train_size   = 0.8,
                                        random_state = 42,
                                        shuffle      = True
                                    )

In [ ]:
# Creación del modelo SVM lineal
# ==============================================================================
modelo = SVC(C = 100, kernel = 'linear', random_state=42)
modelo.fit(X_train, y_train)

Al tratarse de un problema de dos dimensiones, se puede representar las regiones de clasificación.

In [ ]:
# Representación gráfica de los límites de clasificación
# ==============================================================================
# Grid de valores
x = np.linspace(np.min(X_train.X1), np.max(X_train.X1), 50)
y = np.linspace(np.min(X_train.X2), np.max(X_train.X2), 50)
Y, X = np.meshgrid(y, x)
grid = np.vstack([X.ravel(), Y.ravel()]).T

# Predicción valores grid
pred_grid = modelo.predict(grid)

fig, ax = plt.subplots(figsize=(6,4))
ax.scatter(grid[:,0], grid[:,1], c=pred_grid, alpha = 0.2)
ax.scatter(X_train.X1, X_train.X2, c=y_train, alpha = 1)

# Vectores soporte
ax.scatter(
    modelo.support_vectors_[:, 0],
    modelo.support_vectors_[:, 1],
    s=200, linewidth=1,
    facecolors='none', edgecolors='black'
)

# Hiperplano de separación
ax.contour(
    X,
    Y,
    modelo.decision_function(grid).reshape(X.shape),
    colors = 'k',
    levels = [-1, 0, 1],
    alpha  = 0.5,
    linestyles = ['--', '-', '--']
)

ax.set_title("Resultados clasificación SVM lineal");

Se calcula el porcentaje de aciertos que tiene el modelo al predecir las observaciones de test (accuracy).

In [ ]:
# Predicciones test
# ==============================================================================
predicciones = modelo.predict(X_test)
predicciones

In [ ]:
# Accuracy de test del modelo
# ==============================================================================
accuracy = accuracy_score(
            y_true    = y_test,
            y_pred    = predicciones,
            normalize = True
           )
print("")
print(f"El accuracy de test es: {100*accuracy}%")

## SVM radial
Se repite el ajuste del modelo, esta vez empleando un kernel radial y utilizando validación cruzada para identificar el valor óptimo de penalización `C`

- `C` : float, default=1.0
Parámetro de regularización. La fuerza de la regularización es inversamente proporcional a C. Debe ser estrictamente positivo.
    
- `gamma` : **Se usa en modelos no lineales**. Define cuánta curvatura queremos en la frontera de decisión:
  - Gamma alta significa más curvatura.
  - Gamma baja significa menos curvatura.

  El parámetro `gamma` define hasta dónde llega la influencia de un único ejemplo de entrenamiento, donde valores bajos significan 'lejos' y valores altos significan 'cerca'. Los valores más bajos de `gamma` dan como resultado modelos con menor precisión, al igual que los valores más altos de `gamma`. Son los valores intermedios de `gamma` los que dan un modelo con buenos límites de decisión. Valores de gamma:
  - `gamma` = 'scale' el valor será 1/(n_características * X.var())
  - `gamma` = 'auto' el valor será 1/n_características
  - `gamma` = float (no negativo)


- Hay más hiperparámetros, pero estos dos son los importantes:
  - https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html
  - https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html



In [ ]:
# Grid de hiperparámetros
# ==============================================================================
param_grid = {'C': np.logspace(-5, 7, 20)}

# Búsqueda por validación cruzada
# ==============================================================================
grid = GridSearchCV(
        estimator  = SVC(kernel= "rbf", gamma='scale'),
        param_grid = param_grid,
        scoring    = 'accuracy',
        n_jobs     = -1,
        cv         = 3,
        verbose    = 0,
        return_train_score = True
      )

# Se asigna el resultado a _ para que no se imprima por pantalla
_ = grid.fit(X = X_train, y = y_train)

# Resultados del grid
# ==============================================================================
resultados = pd.DataFrame(grid.cv_results_)
resultados.filter(regex = '(param.*|mean_t|std_t)')\
    .drop(columns = 'params')\
    .sort_values('mean_test_score', ascending = False) \
    .head(5)

In [ ]:
# Mejores hiperparámetros por validación cruzada
# ==============================================================================
print("----------------------------------------")
print("Mejores hiperparámetros encontrados (cv)")
print("----------------------------------------")
print(grid.best_params_, ":", grid.best_score_, grid.scoring)

modelo = grid.best_estimator_
C_best = grid.best_params_['C']

In [ ]:
# Una función para representar los SVM
# ==============================================================================
def plotSVC(title):
  # Grid de valores
  x = np.linspace(np.min(X_train.X1), np.max(X_train.X1), 50)
  y = np.linspace(np.min(X_train.X2), np.max(X_train.X2), 50)
  Y, X = np.meshgrid(y, x)
  grid = np.vstack([X.ravel(), Y.ravel()]).T

  # Predicción valores grid
  pred_grid = modelo.predict(grid)

  fig, ax = plt.subplots(figsize=(6,4))
  ax.scatter(grid[:,0], grid[:,1], c=pred_grid, alpha = 0.2)
  ax.scatter(X_train.X1, X_train.X2, c=y_train, alpha = 1)

  # Vectores soporte
  ax.scatter(
      modelo.support_vectors_[:, 0],
      modelo.support_vectors_[:, 1],
      s=200, linewidth=1,
      facecolors='none', edgecolors='black'
  )

  # Hiperplano de separación
  ax.contour(
      X,
      Y,
      modelo.decision_function(grid).reshape(X.shape),
      colors='k',
      levels=[0],
      alpha=0.5,
      linestyles='-'
  )

  ax.set_title(title);

In [ ]:
plotSVC("Parámetro gamma por defecto")

Vamos a comprobar el efecto del parámetro gamma. Lo probaremos para varios valores de gamma y fijando el valor de C al encontrado anteriormente.

In [ ]:
gammas = [0.1, 1, 10, 100]
for gamma in gammas:
   modelo = SVC(kernel='rbf', C=C_best, gamma=gamma).fit(X_train, y_train)
   plotSVC('gamma=' + str(gamma))
   predicciones = modelo.predict(X_test)
   accuracy = accuracy_score(
            y_true    = y_test,
            y_pred    = predicciones,
            normalize = True
           )
   print("")
   print(f"El accuracy de test es: {100*accuracy}% con gamma " + str(gamma))

El mejor modelo es con un valor de gamma de 1.

In [ ]:
modelo = SVC(kernel='rbf', C=C_best, gamma=1).fit(X_train, y_train)

In [ ]:
# Predicciones test
# ==============================================================================
predicciones = modelo.predict(X_test)

In [ ]:
# Accuracy de test del modelo
# ==============================================================================
accuracy = accuracy_score(
            y_true    = y_test,
            y_pred    = predicciones,
            normalize = True
           )
print("")
print(f"El accuracy de test es: {100*accuracy}%")

In [ ]:
# Matriz de confusión de las predicciones de test
# ==============================================================================
confusion_matrix = pd.crosstab(
    y_test.ravel(),
    predicciones,
    rownames=['Real'],
    colnames=['Predicción']
)
confusion_matrix

Con un modelo SVM de kernel radial se consigue clasificar correctamente el 85% de las observaciones de test.

In [ ]:
# Entregamos el modelo final entrenado con todos los datos
modelo_final = SVC(kernel='rbf', C=C_best, gamma=1).fit(X0, y0)

# SVM como regresor

Primero, se cargan los datos, las entradas van a X, las salidas a y.

In [ ]:
# Datos
# ==============================================================================
datos = pd.read_csv('./Student_Marks.csv')
datos.head(3)

Queremos buscar una función que en función de las horas diarias de estudio, nos estime la nota media que vamos a sacar.

In [ ]:
train, test = train_test_split(datos, test_size=0.2, random_state=42)

# train y test datasets los ordenamos para poder dibujarlo bien
train = train.sort_values('time_study')
test = test.sort_values('time_study')

X_train, X_test = train[['time_study']], test[['time_study']]
y_train, y_test = train['Marks'], test['Marks']

X = datos.iloc[:,1:2].values.astype(float)
y = datos.iloc[:,2:3].values.astype(float)


In [ ]:
y_train.describe(include="all")

Comprobamos el rango de la variable de salida va de 5.6 a 53.25


##SVM lineal

En primer lugar, definamos nuestra función python para el RMSE

In [ ]:
from sklearn import metrics

def rmse(y_test, y_test_pred):
  """ Este es mi cálculo del error cuadrático medio """
  return np.sqrt(metrics.mean_squared_error(y_test, y_test_pred))

En primer lugar, obtenemos el modelo lineal con hiperparámetros por defecto

In [ ]:
from sklearn import metrics
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

svr_lineal = SVR(kernel="linear")
escalar = StandardScaler()

pipe_regr_lineal = Pipeline([
    ('escalar', escalar),
    ('SVM', svr_lineal)])

np.random.seed(42)
pipe_regr_lineal.fit(X=X_train, y=y_train)
print(f"RMSE of SVR with default hyper-pars: {rmse(y_test, pipe_regr_lineal.predict(X=X_test))}")
print(f"Param C: {pipe_regr_lineal['SVM'].C}")


Hacemos una búsqueda por el mejor parámetro C con una GridSearch

In [ ]:
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.model_selection import KFold


param_grid = {'SVM__C': [0.1, 1, 10, 100]}


inner = KFold(n_splits=3, shuffle=True, random_state=42)

hpo_regr_lineal = GridSearchCV(pipe_regr_lineal,
                        param_grid,
                        scoring='neg_mean_squared_error',
                        cv=inner,
                        n_jobs=4, verbose=1)

# Train the self-adjusting process
np.random.seed(42)
hpo_regr_lineal.fit(X=X_train, y=y_train)


Visualicemos:
- Los mejores hiperparámetros y su puntuación (¡inner!).
- La evaluación del modelo (¡outer!) con los datos de test y los mejores hiperparámetros.



In [ ]:
print(f"Best params: {hpo_regr_lineal.best_params_}, best score (inner!): {np.sqrt(-hpo_regr_lineal.best_score_)}")
# Now, the performance of regr is computed on the test partition
print(f"RMSE (outer!) of SVR with hyper-parameter tuning (grid-search): {rmse(y_test, hpo_regr_lineal.predict(X=X_test))}")

Observamos que el mejor valor de C es 100 que está en el límite del espacio de búsqueda. Podemos plantearnos ampliar el espacio de búsqueda y ver si mejoran los resultados.

In [ ]:
# Search space
param_grid = {'SVM__C': [0.001, 0.01, 1, 10, 100, 1000, 10000, 50000, 100000]}

hpo_regr_lineal = GridSearchCV(pipe_regr_lineal,
                        param_grid,
                        scoring='neg_mean_squared_error',
                        cv=inner,
                        n_jobs=4, verbose=1)

# Train the self-adjusting process
np.random.seed(42)
hpo_regr_lineal.fit(X=X_train, y=y_train)

In [ ]:
print(f"Best params: {hpo_regr_lineal.best_params_}, best score (inner!): {np.sqrt(-hpo_regr_lineal.best_score_)}")
# Now, the performance of regr is computed on the test partition
print(f"RMSE (outer!) of SVR with hyper-parameter tuning (grid-search): {rmse(y_test, hpo_regr_lineal.predict(X=X_test))}")

##SVM Radial

Pasamos ahora a usar un Kernel radial. Y probamos con los parámetros por defecto

In [ ]:
svr_radial = SVR() #por defecto es el Kernel radial

pipe_regr_radial_def = Pipeline([
    ('escalar', escalar),
    ('SVM', svr_radial)])

np.random.seed(42)
pipe_regr_radial_def.fit(X=X_train, y=y_train)

print(f"Param C: {pipe_regr_radial_def['SVM'].C}, y gamma: {pipe_regr_radial_def['SVM'].gamma}")
print(f"RMSE of SVR with default hyper-pars: {rmse(y_test, pipe_regr_radial_def.predict(X=X_test))}")



## Búsqueda de parámetros en Grid

Hacemos una búsqueda Grid para el mejor parámetro

In [ ]:
param_grid = {'SVM__C': [0.001, 0.01, 1, 10, 100, 1000, 10000, 100000],
              'SVM__gamma': [0.00001, 0.0001, 0.001, 0.01, 0.1, 1]}


inner = KFold(n_splits=3, shuffle=True, random_state=42)

hpo_regr_radial = GridSearchCV(pipe_regr_radial_def,
                               param_grid,
                               scoring='neg_mean_squared_error',
                               cv=inner,
                               n_jobs=4, verbose=1)

# Train the self-adjusting process
np.random.seed(42)
hpo_regr_radial.fit(X=X_train, y=y_train)

In [ ]:
print(f"Best params: {hpo_regr_radial.best_params_}, best score (inner!): {np.sqrt(-hpo_regr_radial.best_score_)}")
# Now, the performance of regr is computed on the test partition
print(f"RMSE (outer!) of SVR with hyper-parameter tuning (grid-search): {rmse(y_test, hpo_regr_radial.predict(X=X_test))}")

### Búsqueda de parámetros en Random

Ahora, vamos a utilizar **Randomized Search** en lugar de gridsearch. Sólo se probarán 20 combinaciones de valores de hiperparámetros (budget=20)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV, KFold
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


budget = 20
hpo_regr_radial = RandomizedSearchCV(pipe_regr_radial_def,
                              param_grid,
                              scoring='neg_mean_squared_error',
                              cv=inner,
                              n_jobs=-1, verbose=1,
                              n_iter=budget
                             )
np.random.seed(42)
hpo_regr_radial.fit(X=X_train, y=y_train)

In [ ]:
print(f"Best params: {hpo_regr_radial.best_params_}, best score (inner!): {np.sqrt(-hpo_regr_radial.best_score_)}")
# Now, the performance of regr is computed on the test partition
print(f"RMSE (outer!) of SVR with hyper-parameter tuning (grid-search): {rmse(y_test, hpo_regr_radial.predict(X=X_test))}")

Hemos obtenido los mismos resultados, pero explorando menos posibilidades que con grid-search.

Para **Randomized Search**, podemos definir el espacio de búsqueda con distribuciones estadísticas, en lugar de utilizar valores particulares como hacíamos antes. A continuación puedes ver cómo utilizar una distribución `loguniform`.


In [ ]:
#from sklearn.utils.fixes import loguniform
from scipy.stats import loguniform


# Search space
param_grid = {'SVM__C': loguniform(1e-1, 1e4),
              'SVM__gamma': loguniform(1e-5, 1e1)}

hpo_regr_radial = RandomizedSearchCV(pipe_regr_radial_def,
                            param_grid,
                            scoring='neg_mean_squared_error',
                            cv=inner,
                            n_jobs=4, verbose=0,
                            n_iter=budget
                        )
np.random.seed(42)
hpo_regr_radial.fit(X=X_train, y=y_train)

In [ ]:
print(f"Best params: {hpo_regr_radial.best_params_}, best score (inner!): {np.sqrt(-hpo_regr_radial.best_score_)}")
# Now, the performance of regr is computed on the test partition
print(f"RMSE (outer!) of SVR with hyper-parameter tuning (grid-search): {rmse(y_test, hpo_regr_radial.predict(X=X_test))}")

## Comparativa entre lineal y radial

Obtenemos y dibujamos las estimaciones para tres casos:


*   Estimador con kernel lineal
*   Estimador con kernel radial óptimo
*   Estimador con kernel radial y parámetro gamma bajo
*   Estimador con kernel radial y parámetro gamma alto



In [ ]:
from matplotlib import pyplot as plt

bestC = hpo_regr_radial.best_params_['SVM__C']

#Usamos un modelo con un gamma baja
pipe_regr_radial_gamma_baja = Pipeline([
    ('escalar', escalar),
    ('SVM', SVR(gamma=0.01))])

#Usamos un modelo con un gamma alta
pipe_regr_radial_gamma_alta = Pipeline([
    ('escalar', escalar),
    ('SVM', SVR(gamma=10))])

np.random.seed(42)
pipe_regr_radial_gamma_baja.fit(X=X_train, y=y_train)
pipe_regr_radial_gamma_alta.fit(X=X_train, y=y_train)

#### Predicciones ####
train['linear_svr_pred'] = hpo_regr_lineal.predict(X_train)
train['rbf_svr_pred_def'] = pipe_regr_radial_def.predict(X_train)
train['rbf_svr_pred_opt'] = hpo_regr_radial.predict(X_train)
train['rbf_svr_pred_baja'] = pipe_regr_radial_gamma_baja.predict(X_train)
train['rbf_svr_pred_alta'] = pipe_regr_radial_gamma_alta.predict(X_train)

#### Visualización ####
plt.scatter(train['time_study'], train['Marks'])
plt.plot(train['time_study'], train['linear_svr_pred'], color = 'orange', label = 'linear SVR')
plt.plot(train['time_study'], train['rbf_svr_pred_def'], color = 'red',  label = 'rbf SVR defecto')
plt.plot(train['time_study'], train['rbf_svr_pred_opt'], color = 'green', linestyle='--', linewidth=3, label = 'rbf SVR óptima')
plt.plot(train['time_study'], train['rbf_svr_pred_baja'], color = 'blue', label = 'rbf SVR gamma baja')
plt.plot(train['time_study'], train['rbf_svr_pred_alta'], color = 'yellow', label = 'rbf SVR gamma alta')
plt.legend()
plt.xlabel('Tiempo de estudio')
plt.ylabel('Puntuación')

Se puede observar en verde como el estimador radial óptimo es el mejor.

## Obtención de los dos modelos para el cliente

Por último, necesitamos un modelo final, podemos obtenerlo entrenando `hpo_regr_radial` a todos los datos disponibles.

In [ ]:
np.random.seed(42)

regrFinal_radial = hpo_regr_radial.fit(X,y)

## Bayesian Optimization (plus)

Para ello se utilizará OPTUNA. **Holdout** para la evaluación del modelo y **3-fold crossvalidation** para el ajuste de hiperparámetros (con **Model Based Optimization** ).


In [ ]:
#Para acceder a diferentes distribuciones y otros métodos
%pip install optuna
#Para acceder a optuna.integration.OptunaSearchCV y permitir la integración con Scikit-Learn
%pip install --upgrade optuna-integration[sklearn]

In [ ]:
import optuna
from optuna.integration import OptunaSearchCV

# Search space
param_grid = {
    'SVM__C': optuna.distributions.FloatDistribution(1e-1, 1e4, log=True),
    'SVM__gamma': optuna.distributions.FloatDistribution(1e-4, 1e0, log=True)
}

hpo_regr_radial = OptunaSearchCV(pipe_regr_radial_def,
                    param_grid,
                    scoring='neg_mean_squared_error',
                    #n_trials=budget,
                    n_trials=50,
                    cv=inner,
                    n_jobs=1, verbose=1,
                    timeout=600,
                    random_state=42
                    )

np.random.seed(42)
hpo_regr_radial.fit(X=X_train, y=y_train)

In [ ]:
print(f"Best params: {hpo_regr_radial.best_params_}, best score (inner!): {np.sqrt(-hpo_regr_radial.best_score_)}")
# Now, the performance of regr is computed on the test partition
print(f"RMSE (outer!) of SVR with hyper-parameter tuning (grid-search): {rmse(y_test, hpo_regr_radial.predict(X=X_test))}")

Podemos comprobar si la optimización ha convergido

In [ ]:
from optuna.visualization.matplotlib import plot_optimization_history
fig = plt.figure(figsize=(25,20))
trial = hpo_regr_radial.study_
plot_optimization_history(trial)
plt.show()

También podemos visualizar la importancia de los distintos parámetros y el modelo final.

In [ ]:
from optuna.visualization.matplotlib import plot_param_importances
fig = plt.figure(figsize=(25,20))
trial = hpo_regr_radial.study_
plot_param_importances(trial)
plt.show()

In [ ]:
from optuna.visualization import plot_parallel_coordinate

optuna_study = hpo_regr_radial.study_

plot_parallel_coordinate(optuna_study)

In [ ]:
from optuna.visualization import plot_contour

plot_contour(optuna_study, params=["SVM__C", "SVM__gamma"])